<img src="https://datasciencecampus.ons.gov.uk/wp-content/uploads/sites/10/2017/03/data-science-campus-logo-new.svg"
             alt="ONS Data Science Campus Logo"
             width = "240"
             style="margin: 0px 60px"
             />

In [19]:
# import the helper functions from the parent directory,
# these help with things like graph plotting and notebook layout
import sys
sys.path.append('..')
from helper_functions import *

# set things like fonts etc - comes from helper_functions
set_notebook_preferences()

# add a show/hide code button - also from helper_functions
toggle_code(title = "import functions")

In [20]:
#dependencies
import pandas as pd
titanic = pd.read_csv('../Data/titanic.csv')

#columns from NewVar script
titanic['child'] = (titanic['age'] < 18).astype(int)
titanic['embarked_city'] = titanic['embarked'].map({'S':'Southampton','C':'Cherbourg','Q':'Queenstown'})
titanic['surname'] = titanic['name'].str.split(',',expand=True)[0]

#columns from descstat script
titanic['z_score'] = (titanic['fare'] - titanic['fare'].mean())/titanic['fare'].std()


toggle_code(title='read data')

# Updating Values


## 9.1 Important! Copies and Views

The selecting of columns and filtering of rows that you've done above effectively **copies** the original dataframe to make a new one based on your conditions. This is great for creating a pared-down dataframe to work with, or when filtering data to produce statistics for particular subsets of data.

However, when we want to actually update cell values we need to use a special function that ensures we are editing the original dataframe **view** and not a **copy** of it. If we edit cells on a copy of a dataframe, we might find that these values are not updated in the original dataframe when we come to analyse it!

This is a technical point that you don't need to worry too much about, just remember to use the following approaches when dealing with code!

In the titanic data, given what we know so far, we might try to update a cell values using what's called 'chained indexing':

```python
titanic[titanic['embarked'] == 'C']['embarked'] = 'c'
```
The above code makes sense - filter the rows, select the column and assign the value you want. However, Python will give you a warning (specifically a 'SettingWithCopyWarning') that you're not doing things properly!

Instead we'll do the following:

```python
titanic.loc[titanic['embarked'] == 'C','embarked'] = 'c'
```

The code looks very similar, however in the second (correct) example we're using `.loc[]`

`.loc[]` is actually almost identical to the selecting and filtering we've already done. We just specify the rows and columns within the square bracket like: `.loc[row_indexer, col_indexer]` where:
* `row_indexer` is the mask if we are filtering, or a colon, `:`, if we want to include all rows.
* `col_indexer` is a single column name, or a list of column names, or a colon, `:`, if we want to include all columns.

You can actually use the .loc[] approach to do all the selecting and filtering we've already done if you want.

In addition to `.loc[]` we also have `.iloc[]` which behaves very similarly, except that it allows us to index on the index position of rows and columns.

Some examples using iloc[]:

In [21]:
# Select all rows, and the first column
titanic.iloc[:,0].head()

0    1
1    1
2    1
3    1
4    1
Name: pclass, dtype: int64

In [22]:
# Or select a range of columns
titanic.iloc[:,3:5].head()

,sex,age
0,female,29.0000
1,male,0.9167
2,female,2.0000
3,male,30.0000
4,female,25.0000


In [23]:
# Another head-like indexing approach
titanic.iloc[0:5,:]

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,child,embarked_city,surname,z_score
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,0,Southampton,Allen,3.439849
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,1,Southampton,Allison,2.284729
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,1,Southampton,Allison,2.284729
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,0,Southampton,Allison,2.284729
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,0,Southampton,Allison,2.284729


Now some examples using `.loc[]`

In [24]:
# Often researchers use column selection to reorder the columns in a dataframe.
cols = list(titanic.columns)
cols.reverse()
titanic.loc[:,cols].head()

,z_score,surname,embarked_city,child,embarked,cabin,fare,ticket,parch,sibsp,age,sex,name,survived,pclass
0,3.439849,Allen,Southampton,0,S,B5,211.3375,24160,0,0,29.0000,female,"Allen, Miss. Elisabeth Walton",1,1
1,2.284729,Allison,Southampton,1,S,C22 C26,151.5500,113781,2,1,0.9167,male,"Allison, Master. Hudson Trevor",1,1
2,2.284729,Allison,Southampton,1,S,C22 C26,151.5500,113781,2,1,2.0000,female,"Allison, Miss. Helen Loraine",0,1
3,2.284729,Allison,Southampton,0,S,C22 C26,151.5500,113781,2,1,30.0000,male,"Allison, Mr. Hudson Joshua Creighton",0,1
4,2.284729,Allison,Southampton,0,S,C22 C26,151.5500,113781,2,1,25.0000,female,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",0,1


In [25]:
# loc takes row conditions as usual.
titanic.loc[titanic['pclass'].isin([1,2]),['pclass','name']].sample(5)

,pclass,name
361,2,"Caldwell, Mrs. Albert Francis (Sylvia Mae Harb..."
252,1,"Ryerson, Mr. Arthur Larned"
259,1,"Seward, Mr. Frederic Kimber"
406,2,"Faunthorpe, Mr. Harry"
576,2,"Veal, Mr. James"


Often pandas users will use chained indexing when selecting and filtering, and the `loc` and `iloc` operators when cleaning values on a cell-by-cell basis. There is no reason why you can't use these operators for selecting and filtering too. If you want to assign the sub-set DataFrame to a new variable, you'll need to use the `.copy()` method to ensure that it is a different dataframe in memory, rather than a reference to the original.
```python
new_df = titanic.loc[:,['survived','embarked']].copy()
```

## 9.2 Propagating Missing Data Values with `.loc[]`

Earlier, when we created a new binary variable called 'child' the condition we used created a `Series` of `True` and `False` values based on whether the condition was met or not.

An important consideration when creating a new variable is the presence of missing data. A condition will automatically set a missing value to `False`. This is fine for filtering data, but misrepresents data in a new variable - it makes a derived variable appear more complete than it is in reality.

In order to update the 'child' column to incorporate rows we know are missing we can use `.loc[]`:
```python
# This was the column originally generated.
titanic['child'] = (titanic['age'] < 18).astype(int)
# Now, update the missing data values
titanic.loc[titanic['age'].isnull(),'child'] = titanic[titanic['age'].isnull()]['age']
```
In the 2nd line of code, we use the row filter `titanic['age'].isnull()` to filter missing data in the 'age' column and we select the 'child' column. These are used to create a 'view' of the titanic DataFrame using `.loc[]`.

Then, we assign to that combination of rows and columns the values given by: `titanic[titanic['age'].isnull()]['age']` which will be a series of missing values. We should now see missing values in the 'child' column.

In [26]:
# initial composition of 'child'
titanic['child'].value_counts(dropna=False)

0    1155
1     154
Name: child, dtype: int64

In [27]:
# update and check new field
titanic.loc[titanic['age'].isnull(),'child'] = titanic[titanic['age'].isnull()]['age']
titanic['child'].value_counts(dropna=False)

0.0    892
NaN    263
1.0    154
Name: child, dtype: int64

## 9.3 Changing Column Data Types

As with basic Python data types, we can cast columns of data from one data type to another. This can be useful as part of a data cleaning process. Sometimes we find that data we expect to be numeric is actually string data, often occurs because the numbers are actually stored as text in the original dataset, in which case they have to be converted. Other times it is because the original dataset includes characters that pandas won't immediately interpret as a numeric value. This is particularly the case if a dataset contains missing data that is coded to a special character. Reading in these data as text is the safest option, as it preserves all of the information and requires the data analyst to make a decision as to how to handle the conversion to a numeric data type.

There are two ways to change a datatype, firstly the series method: `.astype()` in which the parameter is a data type e.g.
```python
titanic['survived'].astype(str)
```
There is also a 'top-level' pandas function `pd.to_numeric()` which is a good option for data cleaning.
```python
pd.to_numeric(titanic['survived'])
```

In [28]:
# turned survived to a string column
titanic['survived'] = titanic['survived'].astype(str)
titanic.dtypes['survived']

dtype('O')

In [29]:
# turned survived back to a numeric
titanic['survived'] = pd.to_numeric(titanic['survived'])
titanic.dtypes['survived']

dtype('int64')

## 9.4 Updating Column Names

We've been using a dataset that has good, descriptive column names. However, sometimes you're faced with columns that have difficult to use names.

* column names might be really long, these are a pain to type out.
* spaces between words, and presence of special characters (e.g. %^&£ etc.) can cause errors. These are commonly encountered in places and peoples' names. 
* column names may be ambiguous or not sufficiently descriptive.

We can use the pandas `.rename()` method to update column names. To update the column names we pass a dictionary to the parameter 'columns', e.g.
```python
df.rename(columns={'two':'new_name'}, inplace=True)
```
For a DataFrame `df`, the `rename` method allows us to change the names of columns based upon a dictionary. Here, the dictionary key `'two'` is the current name of the column, and the dictionary value `'new_name'` is what we want to rename the column.

In [30]:
titanic.rename(columns={'surname':'family_name'}, inplace=True)
titanic.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,child,embarked_city,family_name,z_score
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,0.0,Southampton,Allen,3.439849
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,1.0,Southampton,Allison,2.284729
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,1.0,Southampton,Allison,2.284729
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,0.0,Southampton,Allison,2.284729
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,0.0,Southampton,Allison,2.284729
